# # Assignment HW1 (4. Image Classification)

#New model Class

In [1]:
import torch
import torch.nn as nn
import numpy as np

In [2]:
class CNN(nn.Module):
    def __init__(self, input_size, n_feature1, n_feature2, output_size,param):
        super(CNN, self).__init__()
        
        self.conv_kernel_size = param["conv_kernel_size"]
        self.pooling_kernel_size = param["pooling_kernel_size"]
        self.stride_size = param["stride_size"]
        self.zero_padding = param["zero_padding"]
        self.max_pooling = param["max_pooling"]
        
        if (self.zero_padding==True):
            out_conv=int(np.sqrt(input_size)/(self.pooling_kernel_size **2))
        else:
            n1= (np.sqrt(input_size)-self.conv_kernel_size)/((self.pooling_kernel_size **2)*self.stride_size**2)
            n2=1/((self.pooling_kernel_size**2)*self.stride_size)- (self.conv_kernel_size /(self.pooling_kernel_size *self.stride_size ))
            out_conv=int((n1+n2+(1/self.pooling_kernel_size )))
        
        
        self.out_conv=out_conv
        self.n_feature1 = n_feature1
        self.n_feature2 = n_feature2
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=self.n_feature1,\
                    kernel_size=self.conv_kernel_size, stride=self.stride_size,  padding=int(self.zero_padding*((self.conv_kernel_size-1)/2)))
        
        self.conv2 = nn.Conv2d(self.n_feature1, self.n_feature2,\
                    kernel_size =self.conv_kernel_size, stride=self.stride_size,  padding=int(self.zero_padding*((self.conv_kernel_size-1)/2)))           
        self.fc1 = nn.Linear(self.n_feature2*self.out_conv*self.out_conv, output_size)
        
        
    def forward(self, x, verbose=False):
        x = self.conv1(x)
        x = F.relu(x)
        if(self.max_pooling == True):
            x = F.max_pool2d(x, kernel_size=self.pooling_kernel_size)
        else:
            x = F.avg_pool2d(x, kernel_size=self.pooling_kernel_size)
        x = self.conv2(x)
        x = F.relu(x)
        if(self.max_pooling == True):
            x = F.max_pool2d(x, kernel_size=self.pooling_kernel_size)
        else:
            x = F.avg_pool2d(x, kernel_size=self.pooling_kernel_size)
        print(x.shape)
        x = x.view(-1, self.n_feature2*self.out_conv*self.out_conv)
        x = self.fc1(x)
        x = F.log_softmax(x, dim=1)
        return x

#Zero Padding Model Tensors

In [3]:
input_size  = 28*28   # images are 28x28 pixels
output_size = 10      # there are 10 classes
n_feature1=16
n_feature2=32

In [4]:
param1={"conv_kernel_size": 5,

        "pooling_kernel_size": 2,

        "stride_size": 1,
       
       "zero_padding":True ,
       
        "max_pooling": True}
model1=CNN(input_size,n_feature1,n_feature2,output_size,param1)

In [5]:
model1.parameters

<bound method Module.parameters of CNN(
  (conv1): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (fc1): Linear(in_features=1568, out_features=10, bias=True)
)>

In [6]:
for i in model1.parameters():
    print (i.size())

torch.Size([16, 1, 5, 5])
torch.Size([16])
torch.Size([32, 16, 5, 5])
torch.Size([32])
torch.Size([10, 1568])
torch.Size([10])


#Valid Padding Model Tensors

In [7]:
param2={"conv_kernel_size": 5,

        "pooling_kernel_size": 2,

        "stride_size": 1,
       
       "zero_padding":False ,
       
        "max_pooling": True}
model2=CNN(input_size,n_feature1,n_feature2,output_size,param2)

In [8]:
model2.parameters

<bound method Module.parameters of CNN(
  (conv1): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=512, out_features=10, bias=True)
)>

In [9]:
for i in model2.parameters():
    print (i.size())

torch.Size([16, 1, 5, 5])
torch.Size([16])
torch.Size([32, 16, 5, 5])
torch.Size([32])
torch.Size([10, 512])
torch.Size([10])
